In [ ]:
#Import modules:
import pipistrello
import iris
database_dir = ('/home/juan/MHPC-Thesis/NetCDF_files/')
cubes = iris.load(database_dir+'coads_climatology.nc.nc')
for cube in cubes:
    for coord in cube.coords('TIME'):
        print(coord.name())
        pipistrello.cube_utils.get_bounds(cube,coord.name())

In [1]:
#import pipistrello
import database


Cleaning up compiled files...
compiled_krb: No such file or directory.


In [2]:
database_dir = ('/home/juan/MHPC-Thesis/NetCDF_files')
all_cubes = database.database(database_dir)

CATALOGUE.txt not found in /home/juan/MHPC-Thesis/NetCDF_files.
Trying to create it now...
['catalogue.txt', 'u.nc', 'coords.nc', 'new_mslp.nc', 'mslp.nc', 'fnoc1.nc', 'coords_all.nc', 'coads_climatology.nc.nc']
reading catalogue.txt
Cannot open file /home/juan/MHPC-Thesis/NetCDF_files/catalogue.txt. Please make sure that the file is compatible with Iris.



reading u.nc



1 cubes read from /home/juan/MHPC-Thesis/NetCDF_files/u.nc:
0: Vector wind eastward component / (meter per second) (-- : 4; -- : 5; -- : 6)
reading coords.nc



0 cubes read from /home/juan/MHPC-Thesis/NetCDF_files/coords.nc:
< No cubes >
Available cubes: 
< No cubes >
reading new_mslp.nc



1 cubes read from /home/juan/MHPC-Thesis/NetCDF_files/new_mslp.nc:
0: air_pressure_at_sea_level / (hectopascal) (time: 2; latitude: 6; longitude: 4)
coordinate = time
coordinate = latitude
coordinate = longitude
reading mslp.nc



1 cubes read from /home/juan/MHPC-Thesis/NetCDF_files/mslp.nc:
0: air_pressure_at_sea_level / (hect

writing [compiled_krb]/fc_rules_cf_fc.py
writing [compiled_krb]/compiled_pyke_files.py


In [39]:
#tomorrow start from here!
for i in range(len(all_cubes.cubes)):
    for j in range(len(all_cubes.cubes[i])):
        if 'wind' in all_cubes.cubes[i][j]:
            print(i,all_cubes.cubes[i][j])

0 Vector wind eastward component
4 Vector wind eastward component
4 Vector wind northward component
5 Vector wind eastward component
5 Vector wind northward component


In [ ]:
line1 = "asdf\n"
line2 = "asdf"
print(line1[:-1])
print(line2)



In [ ]:
pipistrello.create_database(database_dir)

In [ ]:
print(all_cubes)

In [ ]:
for c in cubes:
    if c.name() == 'AIR TEMPERATURE':
        c0 = c
print(c0)


In [ ]:
for coord in c0.coords():
    print(coord.name())

In [ ]:
print(c0.coord('latitude'))
print(c0.coord('longitude'))

In [ ]:
#This will give error:
#ValueError: 
#zero not allowed as a reference year, does not exist in Julian or Gregorian calendars

if(0): print(c0.coord('TIME'))

#Why? ...See the error and next box for the solution.

In [ ]:
#Reason for the error above: The reference year is set to zero...
#See...
print(c0.coord('TIME').units)

In [ ]:
#We change the reference year by doing the following:
c0.coord('TIME').units='hour since epoch'
c0t = c0.coord('TIME')
#"epoch" = 1st of january of 1970.
print(c0t.units)


In [ ]:
#Now we can safely print the coordinate:
print(c0.coord('TIME'))

In [ ]:
#As the 'time_origin' attribute is still wrong...
print(c0.coord('TIME').attributes['time_origin'])

#We change it by:
c0.coord('TIME').attributes['time_origin'] = '01-JAN-1970 00:00:00'

print(c0.coord('TIME').attributes['time_origin'])

In [ ]:
#We need to extract a 1D cube and for this we create constraints:
print("The original cube:\n {}\n".format(c0))
c1 = c0.extract(iris.Constraint(longitude=45))
print("Extracting a sub-cube with only longitude = 45:\n {}\n".format(c1))
c2 = c1.extract(iris.Constraint(latitude=47))
print("Extracting again a sub-cube with only latitude = 47:\n {}\n".format(c2))

#There's another way of extacting a subset of a cube:
#Another way of slicing a cube:
c3 = c0[:,45,80]
print("Extracting at once by indices:\n {}\n".format(c3))

In [ ]:
#If we print c2.data we get something weird:
if(1): print(c2.data)

#That is because cube thata is a "masked array" object.
print(type(c0.data))

#To show only the valid entries, we use:
print((c0.data.compressed()))

#How many entries are not masked, i.e. valid?
print(c0.data.size)
print(c0.data.compressed().size)

In [ ]:
#That was not a good way of getting a 1D cube. We better take the average.
#Indeed, constraining the cube to just one cell I don't think makes much sense...

#1. We put bound to the cells in order to take an area-average:
if not c0.coord('longitude').has_bounds():
    c0.coord('longitude').guess_bounds()

if not c0.coord('latitude').has_bounds():
    c0.coord('latitude').guess_bounds()

#2. We get the area weights of the cells composing the region:
grid_areas = iris.analysis.cartography.area_weights(c0)

#3. We "collapse" our 2D+Time cube into a 0D+Time by averaging using MEAN aggregator:
c0collapsed = c0.collapsed(['longitude', 'latitude'], iris.analysis.MEAN, weights=grid_areas)

print(c0collapsed)

In [ ]:
#We now want to interpolate between tmin and tmax. 

#We get the bounds:
if not c0.coord('TIME').has_bounds():
    c0.coord('TIME').guess_bounds()
minT = c0.coord('TIME').bounds.min()
maxT = c0.coord('TIME').bounds.max()

print("minT = ",minT)
print("maxT = ",maxT)
print("units = ",c0.coord('TIME').units)

#We set the sample points on the coordinate TIME:
sample_points = [('TIME',np.linspace(minT,maxT))]

#We make a 'Linear' interpolation. The other option is 'Nearest'
c_interp = c0collapsed.interpolate(sample_points, iris.analysis.Linear())

#Notice that the extracted and the interpolated cubes look the same
#except for the number of time points available (12 the original, vs 50 the interpolated)
print("Original collapsed Cube: \n{}\n".format(c0collapsed))
print("Interpolated Cube: \n{}\n".format(c_interp))

In [ ]:
#We now plot the evolution of the temperature:
iplt.plot(c0collapsed,'ro')
iplt.plot(c_interp,'b.')
plt.show()

In [ ]:
#We now do the reverse...

#We get the bounds:
if not c0.coord('longitude').has_bounds():
    c0.coord('longitude').guess_bounds()
minLon = c0.coord('longitude').bounds.min()
maxLon = c0.coord('longitude').bounds.max()

if not c0.coord('latitude').has_bounds():
    c0.coord('latitude').guess_bounds()
minLat = c0.coord('latitude').bounds.min()
maxLat = c0.coord('latitude').bounds.max()

print("minLon = ",minLon)
print("maxLon = ",maxLon)
print("units = ",c0.coord('longitude').units)

print("minLat = ",minLat)
print("maxLat = ",maxLat)
print("units = ",c0.coord('latitude').units)

#We set the sample points on the coordinate TIME:
sample_points = [('longitude',np.linspace(minLon,maxLon)),('latitude',np.linspace(minLat,maxLat))]

#We make a 'Linear' interpolation. The other option is 'Nearest'
c_interpol = c0.interpolate(sample_points, iris.analysis.Linear())
#c_interpol2 = c_interpol.interpolate([('longitude',[45]),('latitude',[47])],iris.analysis.Linear())
#Notice that the extracted and the interpolated cubes look the same
#except for the number of time points available (12 the original, vs 50 the interpolated)
#print("Original collapsed Cube: \n{}\n".format(c0collapsed))
print("Interpolated Cube: \n{}\n".format(c_interpol))
#print("Interpolated Cube2: \n{}\n".format(c_interpol2))

In [ ]:
for coord in c0.coords():
    print(coord.standard_name)
